Imports

In [8]:
import random
import numpy as np
import matplotlib.pyplot as plt

# Rules:
- If $\alpha$ = $\beta$ = 1 then we have uniform distribution
- If $\alpha$ = $\beta$     then we have a symmetric distribution, where x = 1/2
- If $\alpha$ > $\beta$     then the density is right-leaning (concentrated in the neighbourhood of 1) the mean and variance will be able to be computed explicitly



In [9]:
# Gets a Beta Distribution and applies the Thompson Update Rule
def thompson_update_rule(alpha: int = 1, beta: int = 1) -> (int, int):
    sample = np.random.beta(alpha, beta)    # Generate Probability
    r = bandit_sample(sample)               # Get reward
    return alpha+r, beta+(1-r)              # Amend Alpha and Beta values accordingly


# returns reward depending on sample passed
def bandit_sample(p: float) -> int:
    r = random.random()
    return 1 if p > r else 0

# Beta-density plots
Property 1: $\alpha = \beta = 1$

In [10]:
a = 1
b = 1
case_1 = [np.random.beta(a, b) for x in range(0, 100000)]

Property 2: $\alpha = \beta \neq 1$

In [11]:
r = random.random()
while r == 1:
    r = random.random()
case_2 = [np.random.beta(r, r) for x in range(0, 100000)]

Property 3: $\alpha > \beta$